# Calcolo basicness score per tutti i synset di WN
Uno score alto indica che il synset è più basic o generico.

## Utilities

In [38]:
import nltk
from nltk.corpus import wordnet
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso, LassoCV
from sklearn.metrics import root_mean_squared_error, r2_score

In [39]:
scaler = MinMaxScaler()

In [40]:
only_synset_with_agreement = False
select_top_n_features = False

## Dataframe con le features già calcolate

In [41]:
df_features_file = 'features/df_merged_features.csv'
df_merged_features = pd.read_csv(df_features_file)
# print(f"{df_merged_features[:10]}")
print(f"Dimensione df_merged_features: {len(df_merged_features)}")
print(f"Colonne df_merged_features: {df_merged_features.columns.tolist()}")

Dimensione df_merged_features: 82115
Colonne df_merged_features: ['Synset', 'Depth', 'Normalized Depth', 'Agreement score', 'isHard mean', 'timeDiffs mean', 'Normalized timeDiffs mean', 'LCH Similarity', 'Normalized LCH Similarity', 'Frequency', 'Normalized Frequency', 'Synonymy', 'Normalized Synonymy', 'In CBT']


In [42]:
# Rimuovo dal df tutti i synset che non sono nel gold standard
df_merged_features_clean = df_merged_features.copy()
df_merged_features_clean = df_merged_features_clean.dropna(subset=['Agreement score'])
print(f"Dimensione df_merged_features_clean: {len(df_merged_features_clean)}")

Dimensione df_merged_features_clean: 504


In [43]:
agreement_dict = dict(zip(df_merged_features_clean['Synset'], df_merged_features_clean['Agreement score']))
print(agreement_dict)

synset_with_agreement_list = list(df_merged_features_clean['Synset'])
print(synset_with_agreement_list)

{"Synset('whole.n.02')": 1.0, "Synset('motivation.n.01')": 0.9, "Synset('discovery.n.01')": 0.9, "Synset('sleeper.n.09')": 0.5, "Synset('hit.n.03')": 1.0, "Synset('shooting.n.01')": 0.7, "Synset('enfilade.n.01')": 0.0, "Synset('designation.n.03')": 0.0, "Synset('manipulation.n.01')": 0.2, "Synset('mind_game.n.02')": 0.2, "Synset('decision.n.01')": 1.0, "Synset('ruse.n.01')": 0.0, "Synset('means.n.01')": 1.0, "Synset('royal_road.n.01')": 0.3, "Synset('step.n.03')": 1.0, "Synset('trip.n.06')": 1.0, "Synset('motion.n.03')": 0.8, "Synset('inversion.n.08')": 0.7, "Synset('activity.n.01')": 1.0, "Synset('habit.n.02')": 0.9, "Synset('second_nature.n.01')": 0.3, "Synset('dance_step.n.01')": 0.4, "Synset('glissade.n.01')": 0.0, "Synset('position.n.06')": 1.0, "Synset('associateship.n.01')": 0.0, "Synset('care.n.01')": 1.0, "Synset('myringotomy.n.01')": 0.0, "Synset('desensitization_technique.n.01')": 0.0, "Synset('function.n.03')": 0.9, "Synset('behalf.n.01')": 0.0, "Synset('sea-duty.n.01')": 0

Estraiamo le colonne da usare come features.
- Non viene usato 'Agreement score' come feature.

In [44]:
if only_synset_with_agreement:
    columns_names_to_compute_score = ['isHard mean', 'In CBT'] + [col for col in df_merged_features_clean.columns if col.startswith('Normalized')]
else: 
    columns_names_to_compute_score = ['In CBT'] + [col for col in df_merged_features_clean.columns if col.startswith('Normalized') and col != 'Normalized timeDiffs mean']

df_features_to_compute_score = df_merged_features_clean[columns_names_to_compute_score].copy()
# print(df_features_to_compute_score[:5])
print(df_features_to_compute_score.columns.to_list())

['In CBT', 'Normalized Depth', 'Normalized LCH Similarity', 'Normalized Frequency', 'Normalized Synonymy']


## Calcolo dei pesi delle features con ML

### Suddivisione dei dati in training e test set

In [45]:
def get_training_test_sets(X, y, training_size):
    test_size_perc = (100 - training_size)/100
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size_perc, random_state=42)
    
    return X_train, X_test, y_train, y_test

### Models

#### LR - LinearRegression Model

Il modello di regressione lineare viene creato e addestrato senza alcuna ottimizzazione dei suoi parametri.

In [46]:
def compute_linear_regression_model(X, y, training_size):
    X_train, X_test, y_train, y_test = get_training_test_sets(X, y, training_size)
    
    model = LinearRegression()
    
    # Addestra il modello
    model.fit(X_train, y_train)
    
    # Predici sul test set
    predictions = model.predict(X_test)
    
    # Valuta le performance del modello
    mse = root_mean_squared_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    
    return model, mse, r2

##### LR GridSearchCV
Viene utilizzata la GridSearchCV di scikit-learn per cercare i migliori pesi per il modello di regressione lineare. La Grid Search esplora diverse combinazioni di pesi e restituisce il set di pesi che ottimizzano una metrica specifica, come l'errore quadratico medio (MSE) o il coefficiente di determinazione (R²). Viene quindi restituito il modello con i migliori parametri (best_model).

Durante la Grid Search viene utilizzata la cross-validation con 5 fold (cv=5),quindi il training set viene diviso in 5 fold e il modello viene addestrato e validato su ciascun fold in modo iterativo. Ciò aiuta a valutare la performance del modello in modo più robusto.

In [47]:
def compute_linear_regression_model_opt(X, y, training_size):
    X_train, X_test, y_train, y_test = get_training_test_sets(X, y, training_size)
    model = LinearRegression()

    # Definisci i pesi da testare
    param_grid = {
        'fit_intercept': [True, False],
    }

    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='r2')

    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_

    # Predici sul set di test
    predictions = best_model.predict(X_test)
    
    # Valuta le performance del modello
    mse = root_mean_squared_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)

    return best_model, mse, r2

##### LR - CV

Per ottenere una stima più robusta delle performance del modello viene utilizzata la Cross Validation. Viene suddiviso il dataset in più fold e il modello viene addestrato/valutato su diverse combinazioni di dati di training e test.

In [48]:
def compute_linear_regression_model_with_cross_validation(X, y):
    model = LinearRegression()
    model.fit(X, y)

    # CV 5 fold utilizzando MSE
    cv_scores_mse = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')
    cv_scores_mse = -cv_scores_mse
    mean_cv_score_mse = cv_scores_mse.mean()

    # CV 5 fold utilizzando R²
    cv_scores_r2 = cross_val_score(model, X, y, cv=5, scoring='r2')
    mean_cv_score_r2 = cv_scores_r2.mean()
    
    return model, mean_cv_score_mse, mean_cv_score_r2

#### Lasso model
- La regressione lineare cerca di minimizzare solo la somma dei quadrati delle differenze, mentre la regressione Lasso aggiunge una penalità basata sulla somma degli importi assoluti dei coefficienti.
- La regressione Lasso può portare alcuni coefficienti a diventare esattamente zero, fornendo quindi una forma di selezione delle variabili. In altre parole, la regressione Lasso può essere utilizzata per la selezione automatica delle feature, eliminando quelle meno importanti.
- **alpha** è il parametro di **regolarizzazione**, che controlla il grado di riduzione dei pesi. Un valore più alto di alpha porta a una **maggiore riduzione dei pesi** e, quindi, a una maggiore sparsità.

In [49]:
def compute_lasso_model(X, y, training_size):
    X_train, X_test, y_train, y_test = get_training_test_sets(X, y, training_size)
    
    alpha = 0.001
    model = Lasso(alpha=alpha)
    
    model.fit(X_train, y_train)
    
    # Predici sul test set
    predictions = model.predict(X_test)
    
    mse = root_mean_squared_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    
    return model, mse, r2

##### Lasso con CV

È stata utilizzata la Cross Validation (CV) con il modello Lasso per ottenere una stima più robusta delle performance del modello.

In [50]:
def compute_lassoCV_model(X, y):
    model = LassoCV(cv=5)
    model.fit(X, y)
    
    # CV con MSE
    cv_scores_mse = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')
    mean_cv_score_mse = -cv_scores_mse.mean()
    
    # CV con R²
    cv_scores_r2 = cross_val_score(model, X, y, cv=5, scoring='r2')
    mean_cv_score_r2 = cv_scores_r2.mean()

    return model, mean_cv_score_mse, mean_cv_score_r2

### Compute selected model

In [51]:
models = [
    'LinearRegression', 
    'LinearRegression-Cross-Validation', 
    'LinearRegression-Cross-Validation-GridSearchCV',
    'Lasso',
    'LassoCV']

In [52]:
def train_model(selected_model, dataframe, df_features, training_size):
    X = df_features
    y = dataframe['Agreement score']

    model_functions = {
        models[0]: lambda: compute_linear_regression_model(X, y, training_size),
        models[1]: lambda: compute_linear_regression_model_with_cross_validation(X, y),
        models[2]: lambda: compute_linear_regression_model_opt(X, y, training_size),
        models[3]: lambda: compute_lasso_model(X, y, training_size),
        models[4]: lambda: compute_lassoCV_model(X, y),
    }
    
    if selected_model in model_functions:
        model, mse, r2 = model_functions[selected_model]()
        print(f'Model: {model}')
        print(f'Mean Squared Error: {mse}')
        print(f'R-squared: {r2}')
        return model, mse, r2
    else:
        raise ValueError("Model not recognized")

### Pesi delle features

In [53]:
def get_model_weights(selected_model, df_with_agreement, df_features, training_size):
    model, _, _ = train_model(selected_model, df_with_agreement, df_features, training_size)
    
    weights = model.coef_
    
    feature_weights_dict = dict(zip(df_features.columns, weights))
    print("\nFeature Weights:")
    for feature, weight in feature_weights_dict.items():
        print(f"{feature}: {weight}")

    # Normalizzazione dei pesi tra 0 e 1
    weights_normalized = scaler.fit_transform(weights.reshape(-1, 1)).flatten()
    total_weight = sum(weights_normalized)
    weights_normalized /= total_weight
    feature_weights_normalized_dict = dict(zip(df_features.columns, weights_normalized))
    print("\nNormalized Feature Weights:")
    for feature, weight in feature_weights_normalized_dict.items():
        print(f"{feature}: {weight}")
    
    return feature_weights_dict, feature_weights_normalized_dict

In [54]:
selected_model = models[4]
get_model_weights(selected_model, df_merged_features_clean, df_features_to_compute_score, 80)

Model: LassoCV(cv=5)
Mean Squared Error: 0.04957001842386431
R-squared: 0.7212978104397216

Feature Weights:
In CBT: 0.32494178147225666
Normalized Depth: 0.191781665045967
Normalized LCH Similarity: 0.0
Normalized Frequency: 0.576019074999143
Normalized Synonymy: -0.0

Normalized Feature Weights:
In CBT: 0.29736353722288317
Normalized Depth: 0.1755048982441552
Normalized LCH Similarity: 0.0
Normalized Frequency: 0.5271315645329616
Normalized Synonymy: 0.0


({'In CBT': 0.32494178147225666,
  'Normalized Depth': 0.191781665045967,
  'Normalized LCH Similarity': 0.0,
  'Normalized Frequency': 0.576019074999143,
  'Normalized Synonymy': -0.0},
 {'In CBT': 0.29736353722288317,
  'Normalized Depth': 0.1755048982441552,
  'Normalized LCH Similarity': 0.0,
  'Normalized Frequency': 0.5271315645329616,
  'Normalized Synonymy': 0.0})

## Calcolo basicness scores

In [55]:
def compute_basicness_scores(selected_model, df_cleaned, df_features, df_complete, columns_names_to_compute_score, only_synset_with_agreement, training_size):
    model, _, _ = train_model(selected_model, df_cleaned, df_features, training_size)
    
    if not only_synset_with_agreement:
        df_features = df_complete[columns_names_to_compute_score].copy()
        final_df = df_complete
    else:
        final_df = df_cleaned
        
    # Predici sul dataset completo (non più suddividendo in train e test set)
    final_df['Basicness score'] = model.predict(df_features)
    
    # Normalizzazione
    final_df['Basicness score'] = scaler.fit_transform(final_df[['Basicness score']])
    
    return final_df

In [56]:
selected_model = models[4]

df_basicness_scores = compute_basicness_scores(selected_model, df_merged_features_clean, df_features_to_compute_score, df_merged_features, columns_names_to_compute_score, only_synset_with_agreement, 80)
print(df_basicness_scores[:5])

Model: LassoCV(cv=5)
Mean Squared Error: 0.04957001842386431
R-squared: 0.7212978104397216
                           Synset  Depth  Normalized Depth  Agreement score   
0           Synset('entity.n.01')      0          1.000000              NaN  \
1  Synset('physical_entity.n.01')      1          0.947368              NaN   
2      Synset('abstraction.n.06')      1          0.947368              NaN   
3            Synset('thing.n.12')      2          0.894737              NaN   
4           Synset('object.n.01')      2          0.894737              NaN   

   isHard mean  timeDiffs mean  Normalized timeDiffs mean  LCH Similarity   
0          NaN             NaN                        NaN        3.637586  \
1          NaN             NaN                        NaN        2.944439   
2          NaN             NaN                        NaN        2.944439   
3          NaN             NaN                        NaN        2.538974   
4          NaN             NaN                   

In [57]:
basicness_scores_dict = dict(zip(df_basicness_scores['Synset'], df_basicness_scores['Basicness score']))

In [58]:
primi_10_valori = {k: basicness_scores_dict[k] for k in list(basicness_scores_dict)[:10]}
print(primi_10_valori)

{"Synset('entity.n.01')": 0.8836042933890997, "Synset('physical_entity.n.01')": 0.5151537719458408, "Synset('abstraction.n.06')": 0.5252568871817799, "Synset('thing.n.12')": 0.9905890401394974, "Synset('object.n.01')": 0.9905890401394974, "Synset('whole.n.02')": 0.9811780802789947, "Synset('congener.n.03')": 0.13321149391212883, "Synset('living_thing.n.01')": 0.6688066305700457, "Synset('organism.n.01')": 0.24353558321234836, "Synset('benthos.n.02')": 0.11452977187045935}


## Punteggio di basicness per un synset specifico:

In [59]:
synset = str(wordnet.synset('whole.n.02'))
score = basicness_scores_dict.get(synset, 'Synset non classificato')
print(score)

0.9811780802789947


In [60]:
syn_list1 = ["bank", "lamp", "dog", "wine", "water", "pan", "cloud", "drill", "entity", "relation"]
syn_list2 = ["entity", "life", "storm", "action", "docking", "town", "watt", "capillarity", "work", "taxodiaceae"]
syn_list3 = ["animal", "dress", "pen", "ossuary", "engine", "object", "pulsation", "plant", "enclosure", "guide"]
syn_list4 = ["collection", "taxodiaceae", "drill", "perennial", "cat", "tree", "biont", "dog", "book", "can"]
syn_list5 = ["abstraction", "heterotroph", "organism", "pine", "animal", "soup", "benthos", "mountain", "poor", "economy"]
syn_list6 = ["physical_entity", "abstraction", "congener", "living_thing", "organism", "benthos", "dwarf", "heterotroph", "biont", "causal_agent"]
selected_syn_list = syn_list6

score_list = []
for word in selected_syn_list:
    synset = str(wordnet.synset(f'{word}.n.01'))
    score = basicness_scores_dict.get(synset, 'Synset non classificato')
    score_list.append(round(score, 2))
    
print(score_list)

[0.52, 0.47, 0.11, 0.67, 0.24, 0.11, 0.6, 0.11, 0.12, 0.22]


## Calcolo threshold
Si desidera calcolare una soglia in base alla quale classificare un termine come 'basic' o 'advanced'.  
Per determinare la soglia viene usato il *gold standard* costituito da 10 annotatori umani che hanno classificato 504 synset.  
Si ha una soglia migliore quanti più synset sono stati classificati allo stesso modo dall'algoritmo rispetto al gold standard.
- La soglia migliore trovata è 0.6990000000000003, con un totale di 467 synset su 504 classificati come nel gold standard.

In [61]:
def compute_threshold(basicness_scores_dict, agreement_dict):
    total_synset_with_agreem = len(agreement_dict.items())
    best_count = 0
    count = 0
    threshold = 0.3
    best_threshold = threshold

    while threshold < 1:
        for syn_with_agreement, agreement_score in agreement_dict.items():
            basicness_score = basicness_scores_dict.get(syn_with_agreement, 'Synset non classificato')
            
            basicness_score_binary = 1 if basicness_score >= threshold else 0
            agreement_score_binary = 1 if agreement_score >= threshold else 0
            
            if agreement_score_binary == basicness_score_binary:
                count += 1
        
        if best_count <= count: 
            best_count = count
            best_threshold = threshold
        
        threshold += 0.001
        count = 0

    if total_synset_with_agreem == best_count:
        print('Tutti i', total_synset_with_agreem, 'synset sono stati classificati correttamente.')
    else:
        print('Su', total_synset_with_agreem, 'synset del gold standard, solo', best_count, 'sono stati classificati correttamente')
    
    return best_threshold

In [62]:
threshold = compute_threshold(basicness_scores_dict, agreement_dict)
print('best threshold found:', threshold)

Su 504 synset del gold standard, solo 467 sono stati classificati correttamente
best threshold found: 0.6990000000000003


In [63]:
def compute_answer(df_basicness_scores, basicness_scores_dict, agreement_dict):
    computed_answers_list = []
    threshold = compute_threshold(basicness_scores_dict, agreement_dict)
    
    for _, computed_score in basicness_scores_dict.items():
        basic_or_advance_computed_score = 'basic' if computed_score >= threshold else 'advanced'
        computed_answers_list.append(basic_or_advance_computed_score)
    
    agreement_answers_list = []
    for agreem_score in df_basicness_scores['Agreement score']:
        if pd.isna(agreem_score):
            basic_or_advance_agreem_score = None
        else:
            basic_or_advance_agreem_score = 'basic' if agreem_score >= threshold else 'advanced'
        agreement_answers_list.append(basic_or_advance_agreem_score)
    return computed_answers_list, agreement_answers_list

Riorganizzazione delle colonne del dataframe

In [64]:
computed_answers_list, agreement_answers_list = compute_answer(df_basicness_scores, basicness_scores_dict, agreement_dict)
df_features_with_answers = df_basicness_scores.copy()


df_columns_names_list = df_features_with_answers.columns.tolist()

df_columns_names_list.remove('Agreement score')
col_index_basicness = df_columns_names_list.index('Basicness score')
df_columns_names_list.insert(col_index_basicness + 1, 'Agreement score')
df_features_with_answers = df_features_with_answers[df_columns_names_list]

index_column_agreement = df_columns_names_list.index('Agreement score')
df_features_with_answers.insert(index_column_agreement, 'Agreement answer', agreement_answers_list)
df_features_with_answers.insert(col_index_basicness, 'Basicness score answer', computed_answers_list)

Su 504 synset del gold standard, solo 467 sono stati classificati correttamente


In [65]:
print(df_features_with_answers[:5])

                           Synset  Depth  Normalized Depth  isHard mean   
0           Synset('entity.n.01')      0          1.000000          NaN  \
1  Synset('physical_entity.n.01')      1          0.947368          NaN   
2      Synset('abstraction.n.06')      1          0.947368          NaN   
3            Synset('thing.n.12')      2          0.894737          NaN   
4           Synset('object.n.01')      2          0.894737          NaN   

   timeDiffs mean  Normalized timeDiffs mean  LCH Similarity   
0             NaN                        NaN        3.637586  \
1             NaN                        NaN        2.944439   
2             NaN                        NaN        2.944439   
3             NaN                        NaN        2.538974   
4             NaN                        NaN        2.538974   

   Normalized LCH Similarity  Frequency  Normalized Frequency  Synonymy   
0                   1.000000   0.000013              0.765746         1  \
1             

## Salvataggio su file del dataframe aggiornato con lo score di basicness

In [66]:
"""
sorted_normalized_basicness_scores = dict(sorted(basicness_scores_dict.items(), key=lambda item: item[1], reverse=True))

file_name = 'basicness_scores_with_agreement.txt'
with open(file_name, 'w') as file:
    for key, value in sorted_normalized_basicness_scores.items():
        file.write(str(key))
        file.write('\n')
        file.write(str(value))
        file.write('\n\n')
"""

"\nsorted_normalized_basicness_scores = dict(sorted(basicness_scores_dict.items(), key=lambda item: item[1], reverse=True))\n\nfile_name = 'basicness_scores_with_agreement.txt'\nwith open(file_name, 'w') as file:\n    for key, value in sorted_normalized_basicness_scores.items():\n        file.write(str(key))\n        file.write('\n')\n        file.write(str(value))\n        file.write('\n\n')\n"

In [67]:
df_file_output = 'df_basicness_score.csv'
df_features_with_answers.to_csv(df_file_output, index=False)

# Risultati ottenuti

In [68]:
print(df_features_with_answers[:10])

                           Synset  Depth  Normalized Depth  isHard mean   
0           Synset('entity.n.01')      0          1.000000          NaN  \
1  Synset('physical_entity.n.01')      1          0.947368          NaN   
2      Synset('abstraction.n.06')      1          0.947368          NaN   
3            Synset('thing.n.12')      2          0.894737          NaN   
4           Synset('object.n.01')      2          0.894737          NaN   
5            Synset('whole.n.02')      3          0.842105          0.0   
6         Synset('congener.n.03')      4          0.789474          NaN   
7     Synset('living_thing.n.01')      4          0.789474          NaN   
8         Synset('organism.n.01')      5          0.736842          NaN   
9          Synset('benthos.n.02')      6          0.684211          NaN   

   timeDiffs mean  Normalized timeDiffs mean  LCH Similarity   
0             NaN                        NaN        3.637586  \
1             NaN                        NaN  

## Numero di synset classificati come 'ADVANCED' e come 'BASIC'.

In [69]:
def count_basic_advanced(dataframe):
    count_advanced = 0
    count_basic = 0
    basic_synset_list, advanced_synset_list = [], []
    for val, synset in zip(dataframe['Basicness score answer'], dataframe['Synset']):
        if val == 'advanced':
            count_advanced += 1
            advanced_synset_list.append(synset)
        else:
            count_basic += 1
            basic_synset_list.append(synset)
    
    return count_advanced, count_basic, advanced_synset_list, basic_synset_list

num_advanced_syn, num_basic_syn, advanced_synset_list, basic_synset_list = count_basic_advanced(df_features_with_answers)

print("I synset classificati come 'ADVANCED' sono:", num_advanced_syn)
print("I primi 10 synset classificati come 'ADVANCED' sono:", pd.DataFrame(advanced_synset_list)[:10])

print("\nI synset classificati come 'BASIC' sono:", num_basic_syn)
print("I primi 10 synset classificati come 'BASIC' sono:", pd.DataFrame(basic_synset_list)[:10])

I synset classificati come 'ADVANCED' sono: 74394
I primi 10 synset classificati come 'ADVANCED' sono:                                 0
0  Synset('physical_entity.n.01')
1      Synset('abstraction.n.06')
2         Synset('congener.n.03')
3     Synset('living_thing.n.01')
4         Synset('organism.n.01')
5          Synset('benthos.n.02')
6            Synset('dwarf.n.03')
7      Synset('heterotroph.n.01')
8            Synset('biont.n.01')
9     Synset('causal_agent.n.01')

I synset classificati come 'BASIC' sono: 7721
I primi 10 synset classificati come 'BASIC' sono:                        0
0  Synset('entity.n.01')
1   Synset('thing.n.12')
2  Synset('object.n.01')
3   Synset('whole.n.02')
4  Synset('parent.n.02')
5    Synset('life.n.10')
6    Synset('cell.n.02')
7  Synset('person.n.01')
8  Synset('animal.n.01')
9   Synset('plant.n.02')


In [70]:
valori_non_vuoti = df_features_with_answers['Agreement score'].notnull()
count_syn_with_agreement = valori_non_vuoti.sum()
print('Il numero di synset annotati nel gold standard è:', count_syn_with_agreement)

Il numero di synset annotati nel gold standard è: 504


## Numero di synset non classificati come nel gold standard

In [71]:
def get_syn_with_wrong_answer(dataframe):
    syn_with_wrong_answer_dict = {}
    for _, riga in dataframe.iterrows():
        answer_agreement = riga['Agreement answer']
        answer_score = riga['Basicness score answer']
        if pd.isna(answer_agreement):
            pass
        elif answer_agreement != answer_score:
            synset = riga['Synset']
            agreem_score = riga['Agreement score']
            basicness_score = riga['Basicness score']
            time_diff = riga['timeDiffs mean']
            is_hard_mean = riga['isHard mean']
            syn_with_wrong_answer_dict[synset] = (agreem_score, basicness_score, time_diff, is_hard_mean)
    return syn_with_wrong_answer_dict

syn_with_wrong_answer_dict = get_syn_with_wrong_answer(df_features_with_answers)
print("Il numero di synset non classificati come nel gold standard è:", len(syn_with_wrong_answer_dict))
df_syn_with_wrong_answer = pd.DataFrame(list(syn_with_wrong_answer_dict.values()), 
                                        index=syn_with_wrong_answer_dict.keys(), 
                                        columns=['Basicness score', 'Agreement score', 'timeDiffs mean', 'isHard mean'])
print(df_syn_with_wrong_answer)

# salvataggio su file
df_syn_with_wrong_answer.insert(0, 'Synset', df_syn_with_wrong_answer.index)
df_file_output = 'df_syn_with_wrong_answer.csv'
df_syn_with_wrong_answer.to_csv(df_file_output, index=False)


Il numero di synset non classificati come nel gold standard è: 37
                              Basicness score  Agreement score   
Synset('motivation.n.01')                 0.9         0.658338  \
Synset('means.n.01')                      1.0         0.649985   
Synset('inversion.n.08')                  0.7         0.150812   
Synset('behalf.n.01')                     0.0         0.915301   
Synset('contribution.n.01')               0.3         0.943534   
Synset('workshop.n.02')                   0.5         0.836356   
Synset('homo.n.02')                       0.7         0.173856   
Synset('avenue.n.02')                     0.4         0.924712   
Synset('can.n.01')                        1.0         0.640574   
Synset('fabric.n.01')                     0.5         0.799163   
Synset('determination.n.02')              0.6         0.877051   
Synset('human_body.n.01')                 0.8         0.649985   
Synset('convention.n.02')                 0.4         0.962356   
Synset('es

### Numero di synset non classificati come nel gold standard che sono HARD.

In [72]:
def find_hard_synsets(syn_with_wrong_answer_dict, threshold):
    hard_synsets = []
    for synset, values in syn_with_wrong_answer_dict.items():
        is_hard_mean = values[3]

        if is_hard_mean >= threshold:
            hard_synsets.append(synset)

    return hard_synsets

def count_hard_synsets(syn_with_wrong_answer_dict, threshold):
    hard_synsets = find_hard_synsets(syn_with_wrong_answer_dict, threshold)
    return len(hard_synsets), hard_synsets

num_hard_synsets, hard_synsets_list = count_hard_synsets(syn_with_wrong_answer_dict, threshold)
print(f"Il numero di synset considerati 'hard' (con isHard mean >= {threshold}) è:", num_hard_synsets)
print("Lista di synset considerati 'hard':")
print(hard_synsets_list)

Il numero di synset considerati 'hard' (con isHard mean >= 0.6990000000000003) è: 0
Lista di synset considerati 'hard':
[]


### Valori di isHard mean e timeDiffs mean di un synset specifico

In [73]:
def get_synset_difficulty(syn_with_wrong_answer_dict, synset):
    if synset in syn_with_wrong_answer_dict:
        values = syn_with_wrong_answer_dict[synset]
        is_hard_mean = values[3]
        time_diff_mean = values[2]
        return is_hard_mean, time_diff_mean
    else:
        return None, None

synset = "Synset('motivation.n.01')"
is_hard_mean_value, time_diff_mean_value = get_synset_difficulty(syn_with_wrong_answer_dict, synset)

if is_hard_mean_value is not None and time_diff_mean_value is not None:
    print(f"Il valore di 'isHard mean' per il '{synset}' è:", is_hard_mean_value)
    print(f"Il {synset} è stato quindi valutato come {'HARD' if is_hard_mean_value >= threshold else 'NON HARD'} in base alla soglia {threshold}")
    print(f"\nIl valore di 'timeDiffs mean' per il synset '{synset}' è:", time_diff_mean_value)
else:
    print(f"Il {synset} non è un synset con agreement umano.")

Il valore di 'isHard mean' per il 'Synset('motivation.n.01')' è: 0.0
Il Synset('motivation.n.01') è stato quindi valutato come NON HARD in base alla soglia 0.6990000000000003

Il valore di 'timeDiffs mean' per il synset 'Synset('motivation.n.01')' è: 1.6116999999999997


# **STATISTICHE**

PESI normalizzati DELLE FEATURES con only_synset_with_agreement = **False**

- models[0]
    - In CB corpus: 0.20055614247529385
    - Normalized Depth: 0.0
    - Normalized LCH Similarity: 0.4542789375493259
    - Normalized Frequency: 0.23024920502670743
    - Normalized Polysemy: 0.11491571494867293

- models[1]
    - In CB corpus: 0.19824315027636746
    - Normalized Depth: 0.0
    - Normalized LCH Similarity: 0.4419459557938856
    - Normalized Frequency: 0.23795853172772272
    - Normalized Polysemy: 0.12185236220202411

- models[2]
    - In CB corpus: 0.20069157849061653
    - Normalized Depth: 0.0
    - Normalized LCH Similarity: 0.4543585753964545
    - Normalized Frequency: 0.23043251993560798
    - Normalized Polysemy: 0.11451732617732088

- models[3] aplha 0.01
    - In CB corpus: 0.3793134122963464
    - Normalized Depth: 0.0
    - Normalized LCH Similarity: 0.0
    - Normalized Frequency: 0.6206865877036536
    - Normalized Polysemy: 0.0

- models[3] aplha 0.001
    - In CB corpus: 0.2958827130997813
    - Normalized Depth: 0.1995377800150995
    - Normalized LCH Similarity: 0.05309652386011924
    - Normalized Frequency: 0.451482983025
    - Normalized Polysemy: 0.0

- models[4]
    - In CB corpus: 0.29736353722288317
    - Normalized Depth: 0.1755048982441552
    - Normalized LCH Similarity: 0.0
    - Normalized Frequency: 0.5271315645329616
    - Normalized Polysemy: 0.0